In [46]:
import pandas as pd
import string
import re
from unidecode import unidecode
import numpy as np

df = pd.read_csv(r"Arquivo_limpar (1).csv", sep=";", encoding='latin1')
df_original = df.copy()
df

,Cod Cliente,Nome,DocId (11 digitos),endereco,seq
0,CC000001,Cliente 1,555,"RUA 10 DE MAIO, 101 - tt",1
1,CC001124,Cliente 1124,1623,"Rua Eudoria Xavier da Silva, 3231 - casa",2
2,CC007704,Cliente 7704,79261,"RUA PRINCIPAL, 2 - CONGONHAL",3
3,CC007727,Cliente 7727,79284,"BRAZILIANO BRANDAO , 70 -",4
4,CC000001,Cliente 1,3558,"RUA ISMENEA MARIA DOS SANTOS , 445 -",5
...,...,...,...,...,...
8646,CC008648,Cliente 8648,960555,"CUMBICA , - 44",8648
8647,CC008649,Cliente 8649,960565,"JANDYRA R MACHADO DA ROSA, - 0,166666666666667",8649
8648,CC008650,Cliente 8650,960575,"RUA 16, - 44 C",8650
8649,CC008651,Cliente 8651,960585,"RUA JOSE CORREA ANTUNES, - 3434",8651


Tarefa 1 - limpar/normalizar:
Separar o endereço criando novos campos: TIPO_LOGRADOURO, NOME_LOGRADOURO, NUMERO e COMPLEMENTO
*No campo NUMERO deverá conter apenas dados de tipo numéricos, outras informações deverão ser colocadas no campo COMPLEMENTO.

Tarefa 2 - juntar endereços utilizando o DocId
Identificar os Clientes com mesmo Codigo e DocId colocar na mesma linha os endereços encontrados
*Podem ser criados novos campos: (TIPO_LOGRADOURO_1, NOME_LOGRADOURO_1, NUMERO_1, COMPLEMENTO_1; TIPO_LOGRADOURO_2, NOME_LOGRADOURO_2, NUMERO_2, COMPLEMENTO_2; ... ;TIPO_LOGRADOURO_N, NOME_LOGRADOURO_N, NUMERO_N, COMPLEMENTO_N;)

Tarefa 3 - achar/tratar DocId vazios/zerados  
Identificar nome e código de clientes com DocId vazios/zerados utilizando as chaves que julgar ser suficiente para tal (Identificar DocId em novo campo DOCId_SUGERIDO)
Não esquecer de voltar na tarefa 2 e adicionar os endereços referentes ao docs sugeridos.

4 - Enviar arquivo xlsx com os tratamentos executados e prints do código/ferramenta utilizado para fazer os tratamentos.

### Tarefa 1 - limpar/normalizar: 
Separar o endereço criando novos campos: TIPO_LOGRADOURO, NOME_LOGRADOURO, NUMERO e COMPLEMENTO *No campo NUMERO deverá conter apenas dados de tipo numéricos, outras informações deverão ser colocadas no campo COMPLEMENTO.

In [47]:
# Utilizei o sistema de busca do Código de Endereçamento Postal (CEP), 
# disponibilizado pelos Correios, para reconhecer o que é logradouro. São eles:
lista_logradouros = ['Aeroporto', 'Alameda', 'Área', 'Avenida', 'Campo', 'Chácara', 'Colônia', 'Condomínio', 'Conjunto', 
                     'Distrito', 'Esplanada', 'Estação', 'Estrada', 'Favela', 'Fazenda', 'Feira', 'Jardim', 'Ladeira', 'Lago',
                     'Lagoa', 'Largo', 'Loteamento', 'Morro', 'Núcleo', 'Parque', 'Passarela', 'Pátio', 'Praça', 'Quadra', 
                     'Recanto', 'Residencial', 'Rodovia', 'Rua', 'Setor', 'Sítio', 'Travessa', 'Trecho', 'Trevo', 'Vale', 
                     'Vereda', 'Via', 'Viaduto', 'Viela', 'Vila'] # r onze

def rem_vogal(string):
    return (re.sub("[aeiou]","",string)) 

dicio_identificadores_logradouros = {}

for logradouro in lista_logradouros:
    logradouro_lower = logradouro.lower()
    logradouro_sem_vogal = ''
    logradouro_sem_vogal_aux = rem_vogal(logradouro)
    flag_logradouro_sem_vogal_unico = True
    lista_logradouro_sigla = []
    tamanho_sigla = 2
    lista_logradouros_aux = lista_logradouros.copy()
    lista_logradouros_aux.remove(logradouro)
    for logradouro_aux in lista_logradouros_aux:
        while logradouro[:tamanho_sigla] == logradouro_aux[:tamanho_sigla]:
            tamanho_sigla += 1
        if logradouro_sem_vogal_aux == rem_vogal(logradouro_aux):
            flag_logradouro_sem_vogal_unico = False
 
        
    if len(logradouro) > 4:
        while tamanho_sigla < len(logradouro_lower):
            lista_logradouro_sigla.append(logradouro_lower[:tamanho_sigla])
            tamanho_sigla += 1
        if flag_logradouro_sem_vogal_unico:
            logradouro_sem_vogal = rem_vogal(logradouro_lower)

    logradouro_lower_ascii = unidecode(logradouro_lower) 
    logradouro_sem_vogal_ascii = unidecode(logradouro_sem_vogal) 
    lista_logradouro_sigla_ascii = [unidecode(sigla) for sigla in lista_logradouro_sigla]
    
    set_logradouro = {logradouro_lower, logradouro_lower_ascii, logradouro_sem_vogal, logradouro_sem_vogal_ascii, 
                      *lista_logradouro_sigla, *lista_logradouro_sigla_ascii}
    set_logradouro.discard('')
    dicio_identificadores_logradouros[logradouro] = set_logradouro

dicio_identificadores_logradouros    


{'Aeroporto': {'ae',
  'aer',
  'aero',
  'aerop',
  'aeropo',
  'aeropor',
  'aeroport',
  'aeroporto',
  'rprt'},
 'Alameda': {'al', 'ala', 'alam', 'alame', 'alamed', 'alameda', 'lmd'},
 'Área': {'area', 'área'},
 'Avenida': {'av', 'ave', 'aven', 'aveni', 'avenid', 'avenida', 'vnd'},
 'Campo': {'ca', 'cam', 'camp', 'campo', 'cmp'},
 'Chácara': {'ch',
  'cha',
  'chac',
  'chaca',
  'chacar',
  'chacara',
  'chacr',
  'chá',
  'chác',
  'cháca',
  'chácar',
  'chácara',
  'chácr'},
 'Colônia': {'clon',
  'clôn',
  'col',
  'colo',
  'colon',
  'coloni',
  'colonia',
  'colô',
  'colôn',
  'colôni',
  'colônia'},
 'Condomínio': {'cndmin',
  'cndmín',
  'cond',
  'condo',
  'condom',
  'condomi',
  'condomin',
  'condomini',
  'condominio',
  'condomí',
  'condomín',
  'condomíni',
  'condomínio'},
 'Conjunto': {'cnjnt', 'conj', 'conju', 'conjun', 'conjunt', 'conjunto'},
 'Distrito': {'di',
  'dis',
  'dist',
  'distr',
  'distri',
  'distrit',
  'distrito',
  'dstrt'},
 'Esplanada': {'

In [48]:
def delimitacao_endereco(row, dicio_delimitadores=dicio_identificadores_logradouros):
    endereco = row['endereco'].lower()
    endereco_dividido_por_espaco = endereco.split(' ')
    
    # Tipo Logradouro
    tipo_logradouro_a_verificar = endereco_dividido_por_espaco[0]
    # O código regex abaixo remove qualquer pontuação
    tipo_logradouro_a_verificar = re.sub(r'[^\w\s]', ' ', tipo_logradouro_a_verificar).split(' ')[0]
    tipo_logradouro_encontrado = ''
    for tipo_logradouro, variacoes in dicio_delimitadores.items():
        if tipo_logradouro_a_verificar in variacoes:
            tipo_logradouro_encontrado = tipo_logradouro
            break
    
    if tipo_logradouro_encontrado == '':
        tipo_logradouro_encontrado = 'Rua/Outros' 
    
    # Como 'R.' pode indicar mais de um tipo de logradouro, mas com base na minha pesquisa percebi que o mais provável é 
    # indicar uma rua. Como não consigo ter certeza sobre isso, criei essa categoria para mostrar que
    # não existe certeza na sua classificação, mas que o mais provável é que indique uma rua.
    if tipo_logradouro_a_verificar == 'r':
        tipo_logradouro_encontrado = 'RUA/outros'
    
    
    # Nome Logradouro
    endereco_dividido_por_virgula = endereco.split(',')
    nome_logradouro_a_verificar = string.capwords(endereco_dividido_por_virgula[0])
    nome_logradouro = string.capwords(re.sub(r'[^\w\s]', ' ', nome_logradouro_a_verificar)) 
    
    # Número
    numeros_encontrados = re.findall(r'\d+', endereco)
    try:
        # Rua 11, por exemplo
        if numeros_encontrados[0] in nome_logradouro_a_verificar:
            numero_logradouro = int(numeros_encontrados[1])
        else:
            tamanho_base = 0        
            for n in numeros_encontrados:
                if 6 > len(n) > tamanho_base:
                    numero_logradouro = int(n)
                    tamanho_base = len(n)
        if len(str(numero_logradouro)) > 6:
            numero_logradouro = 0
    except:
        numero_logradouro = 0
    
    # COMPLEMENTO
    complemento = re.sub(' +', ' ', endereco).replace(nome_logradouro_a_verificar.lower(),'')
    complemento = complemento.replace(str(numero_logradouro),'').replace(',','').replace('-','')

    return [tipo_logradouro_encontrado, nome_logradouro, numero_logradouro, complemento] 

    

In [49]:
df_temp = df.apply(delimitacao_endereco, axis=1, result_type='expand')

df['TIPO_LOGRADOURO'] = df_temp.iloc[:,0]
df['NOME_LOGRADOURO'] = df_temp.iloc[:,1]
df['NUMERO']          = df_temp.iloc[:,2]
df['COMPLEMENTO']     = df_temp.iloc[:,3]
# df[df.COMPLEMENTO.str.len() > 10]
# df[df.TIPO_LOGRADOURO == "Rua/Outros"]
df


,Cod Cliente,Nome,DocId (11 digitos),endereco,seq,TIPO_LOGRADOURO,NOME_LOGRADOURO,NUMERO,COMPLEMENTO
0,CC000001,Cliente 1,555,"RUA 10 DE MAIO, 101 - tt",1,Rua,Rua 10 De Maio,101,tt
1,CC001124,Cliente 1124,1623,"Rua Eudoria Xavier da Silva, 3231 - casa",2,Rua,Rua Eudoria Xavier Da Silva,3231,casa
2,CC007704,Cliente 7704,79261,"RUA PRINCIPAL, 2 - CONGONHAL",3,Rua,Rua Principal,2,congonhal
3,CC007727,Cliente 7727,79284,"BRAZILIANO BRANDAO , 70 -",4,Rua/Outros,Braziliano Brandao,70,
4,CC000001,Cliente 1,3558,"RUA ISMENEA MARIA DOS SANTOS , 445 -",5,Rua,Rua Ismenea Maria Dos Santos,445,
...,...,...,...,...,...,...,...,...,...
8646,CC008648,Cliente 8648,960555,"CUMBICA , - 44",8648,Rua/Outros,Cumbica,44,
8647,CC008649,Cliente 8649,960565,"JANDYRA R MACHADO DA ROSA, - 0,166666666666667",8649,Rua/Outros,Jandyra R Machado Da Rosa,0,166666666666667
8648,CC008650,Cliente 8650,960575,"RUA 16, - 44 C",8650,Rua,Rua 16,44,c
8649,CC008651,Cliente 8651,960585,"RUA JOSE CORREA ANTUNES, - 3434",8651,Rua,Rua Jose Correa Antunes,3434,


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8651 entries, 0 to 8650
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Cod Cliente         8651 non-null   object
 1   Nome                8648 non-null   object
 2   DocId (11 digitos)  8651 non-null   int64 
 3   endereco            8651 non-null   object
 4   seq                 8651 non-null   int64 
 5   TIPO_LOGRADOURO     8651 non-null   object
 6   NOME_LOGRADOURO     8651 non-null   object
 7   NUMERO              8651 non-null   int64 
 8   COMPLEMENTO         8651 non-null   object
dtypes: int64(3), object(6)
memory usage: 608.4+ KB


### Tarefa 2  
Juntar endereços utilizando o DocId Identificar os Clientes com mesmo Codigo e DocId colocar na mesma linha os endereços encontrados *Podem ser criados novos campos: (TIPO_LOGRADOURO_1, NOME_LOGRADOURO_1, NUMERO_1, COMPLEMENTO_1; TIPO_LOGRADOURO_2, NOME_LOGRADOURO_2, NUMERO_2, COMPLEMENTO_2; ... ;TIPO_LOGRADOURO_N, NOME_LOGRADOURO_N, NUMERO_N, COMPLEMENTO_N;)

In [67]:
# O(n)
def juntar_enderecos(df, tarefa_2_inicial=True):
    df2 = pd.DataFrame(columns=df.columns).astype({'DocId (11 digitos)': 'int64', 'seq': 'int64', 'NUMERO': 'int64'})
    dicio = {}
    tamanho_original = df.shape[1]
    tamanho_novo = df2.shape[1]
    v = 0
    if tarefa_2_inicial:
        v = 1
    else: 
        df2 = df2.astype({'DOCId_SUGERIDO': 'int64'})
    n = 0
    for x in df.values:
        key = f'{x[3-v]}-{x[0]}'
        if key not in dicio.keys():
            dicio[key] = [n,0]
            x = list(x)
            x += [None] * (tamanho_novo - tamanho_original)
            df2.loc[n] = x
        else:
            dicio[key][1] += 1
            if f"TIPO_LOGRADOURO_{dicio[key][1]}" not in df2.columns:
                df2[f"TIPO_LOGRADOURO_{dicio[key][1]}"] = None
                df2[f"NOME_LOGRADOURO_{dicio[key][1]}"] = None
                df2[f"NUMERO_{dicio[key][1]}"] = None
                df2[f"COMPLEMENTO_{dicio[key][1]}"] = None
                tamanho_novo = df2.shape[1]
                print(f"Conjunto de Colunas Adicionadas: {dicio[key][1]}")
            df2.at[dicio[key][0],[f"TIPO_LOGRADOURO_{dicio[key][1]}"]] = x[6-v]
            df2.at[dicio[key][0],[f"NOME_LOGRADOURO_{dicio[key][1]}"]] = x[7-v]
            df2.at[dicio[key][0],[f"NUMERO_{dicio[key][1]}"]] = int(x[8-v])
            df2.at[dicio[key][0],[f"COMPLEMENTO_{dicio[key][1]}"]] = x[9-v]

        n += 1

    return df2


In [56]:
df_tarefa_2 = juntar_enderecos(df)
df_tarefa_2

Conjunto de Colunas Adicionadas: 1
Conjunto de Colunas Adicionadas: 2


,Cod Cliente,Nome,DocId (11 digitos),endereco,seq,TIPO_LOGRADOURO,NOME_LOGRADOURO,NUMERO,COMPLEMENTO,TIPO_LOGRADOURO_1,NOME_LOGRADOURO_1,NUMERO_1,COMPLEMENTO_1,TIPO_LOGRADOURO_2,NOME_LOGRADOURO_2,NUMERO_2,COMPLEMENTO_2
0,CC000001,Cliente 1,555,"RUA 10 DE MAIO, 101 - tt",1,Rua,Rua 10 De Maio,101,tt,None,None,None,None,None,None,None,None
1,CC001124,Cliente 1124,1623,"Rua Eudoria Xavier da Silva, 3231 - casa",2,Rua,Rua Eudoria Xavier Da Silva,3231,casa,None,None,None,None,None,None,None,None
2,CC007704,Cliente 7704,79261,"RUA PRINCIPAL, 2 - CONGONHAL",3,Rua,Rua Principal,2,congonhal,None,None,None,None,None,None,None,None
3,CC007727,Cliente 7727,79284,"BRAZILIANO BRANDAO , 70 -",4,Rua/Outros,Braziliano Brandao,70,,None,None,None,None,None,None,None,None
4,CC000001,Cliente 1,3558,"RUA ISMENEA MARIA DOS SANTOS , 445 -",5,Rua,Rua Ismenea Maria Dos Santos,445,,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8645,CC008647,Cliente 8647,960545,"RUA SEBASTIAO OSWALDO DE CAMARGO, - FUNDOS",8647,Rua,Rua Sebastiao Oswaldo De Camargo,0,fundos,None,None,None,None,None,None,None,None
8646,CC008648,Cliente 8648,960555,"CUMBICA , - 44",8648,Rua/Outros,Cumbica,44,,None,None,None,None,None,None,None,None
8647,CC008649,Cliente 8649,960565,"JANDYRA R MACHADO DA ROSA, - 0,166666666666667",8649,Rua/Outros,Jandyra R Machado Da Rosa,0,166666666666667,None,None,None,None,None,None,None,None
8648,CC008650,Cliente 8650,960575,"RUA 16, - 44 C",8650,Rua,Rua 16,44,c,None,None,None,None,None,None,None,None


In [58]:
df_tarefa_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8612 entries, 0 to 8649
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Cod Cliente         8612 non-null   object
 1   Nome                8611 non-null   object
 2   DocId (11 digitos)  8612 non-null   int64 
 3   endereco            8612 non-null   object
 4   seq                 8612 non-null   int64 
 5   TIPO_LOGRADOURO     8612 non-null   object
 6   NOME_LOGRADOURO     8612 non-null   object
 7   NUMERO              8612 non-null   int64 
 8   COMPLEMENTO         8612 non-null   object
 9   TIPO_LOGRADOURO_1   33 non-null     object
 10  NOME_LOGRADOURO_1   33 non-null     object
 11  NUMERO_1            33 non-null     object
 12  COMPLEMENTO_1       33 non-null     object
 13  TIPO_LOGRADOURO_2   6 non-null      object
 14  NOME_LOGRADOURO_2   6 non-null      object
 15  NUMERO_2            6 non-null      object
 16  COMPLEMENTO_2       6 no

### Tarefa 3 - achar/tratar DocId vazios/zerados
Identificar nome e código de clientes com DocId vazios/zerados utilizando as chaves que julgar ser suficiente para tal (Identificar DocId em novo campo DOCId_SUGERIDO) Não esquecer de voltar na tarefa 2 e adicionar os endereços referentes ao docs sugeridos.

### Análise exploratória

In [59]:
df[(df['DocId (11 digitos)'].isnull()) | (df['DocId (11 digitos)'] == 0)]

,Cod Cliente,Nome,DocId (11 digitos),endereco,seq,TIPO_LOGRADOURO,NOME_LOGRADOURO,NUMERO,COMPLEMENTO
147,CC000155,Cliente 155,0,"RUA ANTONIO DEL FIOL, 141 - (15)9718-3020",149,Rua,Rua Antonio Del Fiol,9718,141 (15)3020
596,CC000438,Cliente 438,0,"R. MOREIRA DA SILVA , 28 - CASA",598,RUA/outros,R Moreira Da Silva,28,casa
2268,0,NaN,0,"EMIDIO FERRAZ FIUZA, 82 - CASA",2270,Rua/Outros,Emidio Ferraz Fiuza,82,casa
6984,0,NaN,0,"RUA:ALBERTO CECILIATO, 158 -",6986,Rua,Rua Alberto Ceciliato,158,
8025,CC000155,Cliente 155,0,"RUA MARECHAL DEODORO DA FONSECA, 1458 - CASA",8027,Rua,Rua Marechal Deodoro Da Fonseca,1458,casa
8029,CC008049,Cliente 92,0,"RUA EMILIO AUGUSTO MENEZES DA SILVA, 199 -",8031,Rua,Rua Emilio Augusto Menezes Da Silva,199,
8036,CC007983,Cliente 7983,0,"AV. POMPEU REALI, 897 - CASA",8038,Avenida,Av Pompeu Reali,897,casa
8044,CC007983,Cliente 7983,0,"RUA FRANCISCO CATTO, 165 -",8046,Rua,Rua Francisco Catto,165,
8650,0,NaN,0,"SARG JOAO BATISTA MIRANDA, 10 - CASA",8652,Rua/Outros,Sarg Joao Batista Miranda,10,casa


In [60]:
df_docid_nulo = df[(df['DocId (11 digitos)'].isnull()) | (df['DocId (11 digitos)'] == 0)]

for x in df_docid_nulo.seq:
    print('Seq: ',x)
    display(df[df.seq == x])
    display(df[df['DocId (11 digitos)'] == x])
    display(df[df['Nome'] == f'Cliente {x}'])
    display(df[df['Cod Cliente'] == f'CC00{x}'])

    

Seq:  149


,Cod Cliente,Nome,DocId (11 digitos),endereco,seq,TIPO_LOGRADOURO,NOME_LOGRADOURO,NUMERO,COMPLEMENTO
147,CC000155,Cliente 155,0,"RUA ANTONIO DEL FIOL, 141 - (15)9718-3020",149,Rua,Rua Antonio Del Fiol,9718,141 (15)3020


,Cod Cliente,Nome,DocId (11 digitos),endereco,seq,TIPO_LOGRADOURO,NOME_LOGRADOURO,NUMERO,COMPLEMENTO


,Cod Cliente,Nome,DocId (11 digitos),endereco,seq,TIPO_LOGRADOURO,NOME_LOGRADOURO,NUMERO,COMPLEMENTO


,Cod Cliente,Nome,DocId (11 digitos),endereco,seq,TIPO_LOGRADOURO,NOME_LOGRADOURO,NUMERO,COMPLEMENTO


...

Seq:  8652


,Cod Cliente,Nome,DocId (11 digitos),endereco,seq,TIPO_LOGRADOURO,NOME_LOGRADOURO,NUMERO,COMPLEMENTO
8650,0,NaN,0,"SARG JOAO BATISTA MIRANDA, 10 - CASA",8652,Rua/Outros,Sarg Joao Batista Miranda,10,casa


,Cod Cliente,Nome,DocId (11 digitos),endereco,seq,TIPO_LOGRADOURO,NOME_LOGRADOURO,NUMERO,COMPLEMENTO


,Cod Cliente,Nome,DocId (11 digitos),endereco,seq,TIPO_LOGRADOURO,NOME_LOGRADOURO,NUMERO,COMPLEMENTO


,Cod Cliente,Nome,DocId (11 digitos),endereco,seq,TIPO_LOGRADOURO,NOME_LOGRADOURO,NUMERO,COMPLEMENTO


#### Com base na análise exploratória dos dados e no fato de que eu não possuo o conhecimento das regras de negócio, defini o seguinte procedimento para preencher o DocId:

1. Caso o registro não possua Cod Cliente, procure por um registro no qual o seu Cod Cliente contenha o valor do seu seq.
    1. Caso encontre um registro com essas características, utilizar o seu DocId, Cod Cliente e Nome para preencher os campos vazio/zerado.
    2. Caso não encontre, procure por um registro com onde o seu DocId seja igual ao seq do registro com campos vazio, então os preencha como os dados deste.
    3. Caso nenhum desses registros seja encontrado, utilize o seq como base para o Cod Cliente, Nome e DocId.

2. Caso o registro possua Cod Cliente, utilize-o como base para definir o DocId.

In [63]:
def tratamento_docid_nulo(row, df_referencia):
    row_docid = row['DocId (11 digitos)']
    if int(row_docid) == 0 or pd.isna(row_docid):
        row_cod_cliente = row['Cod Cliente']
        if row_cod_cliente == '0' or pd.isna(row_cod_cliente):
            seq_str = str(row['seq'])
            tamanho_seq = len(seq_str)
            registro_encontrado_cod_cli = df_referencia[df_referencia['Cod Cliente'].str[-tamanho_seq:] == seq_str]
            registro_encontrado_docid = df_referencia[df_referencia['DocId (11 digitos)'] == row['seq']]
            if registro_encontrado_cod_cli.shape[0] == 1:
                row['DOCId_SUGERIDO'] = registro_encontrado_cod_cli['DocId (11 digitos)'].values[0]
                row['Cod Cliente'] = registro_encontrado_cod_cli['Cod Cliente'].values[0]
                row['Nome'] = registro_encontrado_cod_cli['Nome'].values[0]
            elif registro_encontrado_docid.shape[0] == 1:
                row['DOCId_SUGERIDO'] = registro_encontrado_docid['DocId (11 digitos)'].values[0]
                row['Cod Cliente'] = registro_encontrado_docid['Cod Cliente'].values[0]
                row['Nome'] = registro_encontrado_docid['Nome'].values[0]
            else:
                row['DOCId_SUGERIDO'] = row['seq']
                row['Cod Cliente'] = "CC{:0>6}".format(seq_str)
                row['Nome'] = "Cliente {}".format(seq_str)
        else:
            row['DOCId_SUGERIDO'] = int(row['Cod Cliente'].replace('CC','')) 
        
    return row          

In [64]:
if 'DOCId_SUGERIDO' not in df.columns:
    df.insert(3, "DOCId_SUGERIDO", df['DocId (11 digitos)'])
df = df.apply(lambda row: tratamento_docid_nulo(row,df_original), axis=1, result_type='broadcast')
df

,Cod Cliente,Nome,DocId (11 digitos),DOCId_SUGERIDO,endereco,seq,TIPO_LOGRADOURO,NOME_LOGRADOURO,NUMERO,COMPLEMENTO
0,CC000001,Cliente 1,555,555,"RUA 10 DE MAIO, 101 - tt",1,Rua,Rua 10 De Maio,101,tt
1,CC001124,Cliente 1124,1623,1623,"Rua Eudoria Xavier da Silva, 3231 - casa",2,Rua,Rua Eudoria Xavier Da Silva,3231,casa
2,CC007704,Cliente 7704,79261,79261,"RUA PRINCIPAL, 2 - CONGONHAL",3,Rua,Rua Principal,2,congonhal
3,CC007727,Cliente 7727,79284,79284,"BRAZILIANO BRANDAO , 70 -",4,Rua/Outros,Braziliano Brandao,70,
4,CC000001,Cliente 1,3558,3558,"RUA ISMENEA MARIA DOS SANTOS , 445 -",5,Rua,Rua Ismenea Maria Dos Santos,445,
...,...,...,...,...,...,...,...,...,...,...
8646,CC008648,Cliente 8648,960555,960555,"CUMBICA , - 44",8648,Rua/Outros,Cumbica,44,
8647,CC008649,Cliente 8649,960565,960565,"JANDYRA R MACHADO DA ROSA, - 0,166666666666667",8649,Rua/Outros,Jandyra R Machado Da Rosa,0,166666666666667
8648,CC008650,Cliente 8650,960575,960575,"RUA 16, - 44 C",8650,Rua,Rua 16,44,c
8649,CC008651,Cliente 8651,960585,960585,"RUA JOSE CORREA ANTUNES, - 3434",8651,Rua,Rua Jose Correa Antunes,3434,


In [68]:
df = df.astype({'DocId (11 digitos)': 'int64', 'seq': 'int64', 'NUMERO': 'int64', 'DOCId_SUGERIDO': 'int64'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8651 entries, 0 to 8650
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Cod Cliente         8651 non-null   object
 1   Nome                8651 non-null   object
 2   DocId (11 digitos)  8651 non-null   int64 
 3   DOCId_SUGERIDO      8651 non-null   int64 
 4   endereco            8651 non-null   object
 5   seq                 8651 non-null   int64 
 6   TIPO_LOGRADOURO     8651 non-null   object
 7   NOME_LOGRADOURO     8651 non-null   object
 8   NUMERO              8651 non-null   int64 
 9   COMPLEMENTO         8651 non-null   object
dtypes: int64(4), object(6)
memory usage: 676.0+ KB


### Executando a tarefa 2 novamente, com a base de dados atualizada

In [69]:
df = juntar_enderecos(df, tarefa_2_inicial=False)
df

Conjunto de Colunas Adicionadas: 1
Conjunto de Colunas Adicionadas: 2


,Cod Cliente,Nome,DocId (11 digitos),DOCId_SUGERIDO,endereco,seq,TIPO_LOGRADOURO,NOME_LOGRADOURO,NUMERO,COMPLEMENTO,TIPO_LOGRADOURO_1,NOME_LOGRADOURO_1,NUMERO_1,COMPLEMENTO_1,TIPO_LOGRADOURO_2,NOME_LOGRADOURO_2,NUMERO_2,COMPLEMENTO_2
0,CC000001,Cliente 1,555,555,"RUA 10 DE MAIO, 101 - tt",1,Rua,Rua 10 De Maio,101,tt,None,None,None,None,None,None,None,None
1,CC001124,Cliente 1124,1623,1623,"Rua Eudoria Xavier da Silva, 3231 - casa",2,Rua,Rua Eudoria Xavier Da Silva,3231,casa,None,None,None,None,None,None,None,None
2,CC007704,Cliente 7704,79261,79261,"RUA PRINCIPAL, 2 - CONGONHAL",3,Rua,Rua Principal,2,congonhal,None,None,None,None,None,None,None,None
3,CC007727,Cliente 7727,79284,79284,"BRAZILIANO BRANDAO , 70 -",4,Rua/Outros,Braziliano Brandao,70,,None,None,None,None,None,None,None,None
4,CC000001,Cliente 1,3558,3558,"RUA ISMENEA MARIA DOS SANTOS , 445 -",5,Rua,Rua Ismenea Maria Dos Santos,445,,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8646,CC008648,Cliente 8648,960555,960555,"CUMBICA , - 44",8648,Rua/Outros,Cumbica,44,,None,None,None,None,None,None,None,None
8647,CC008649,Cliente 8649,960565,960565,"JANDYRA R MACHADO DA ROSA, - 0,166666666666667",8649,Rua/Outros,Jandyra R Machado Da Rosa,0,166666666666667,None,None,None,None,None,None,None,None
8648,CC008650,Cliente 8650,960575,960575,"RUA 16, - 44 C",8650,Rua,Rua 16,44,c,None,None,None,None,None,None,None,None
8649,CC008651,Cliente 8651,960585,960585,"RUA JOSE CORREA ANTUNES, - 3434",8651,Rua,Rua Jose Correa Antunes,3434,,None,None,None,None,None,None,None,None


### 4 - Enviar arquivo xlsx com os tratamentos executados e prints do código/ferramenta utilizado para fazer os tratamentos.

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8612 entries, 0 to 8650
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Cod Cliente         8612 non-null   object
 1   Nome                8612 non-null   object
 2   DocId (11 digitos)  8612 non-null   int64 
 3   DOCId_SUGERIDO      8612 non-null   int64 
 4   endereco            8612 non-null   object
 5   seq                 8612 non-null   int64 
 6   TIPO_LOGRADOURO     8612 non-null   object
 7   NOME_LOGRADOURO     8612 non-null   object
 8   NUMERO              8612 non-null   int64 
 9   COMPLEMENTO         8612 non-null   object
 10  TIPO_LOGRADOURO_1   34 non-null     object
 11  NOME_LOGRADOURO_1   34 non-null     object
 12  NUMERO_1            34 non-null     object
 13  COMPLEMENTO_1       34 non-null     object
 14  TIPO_LOGRADOURO_2   5 non-null      object
 15  NOME_LOGRADOURO_2   5 non-null      object
 16  NUMERO_2            5 no

In [71]:
df.to_excel("desafio_tecnico.xlsx", index=False)